# <center> Baseline ResNet

Nous allons utiliser ResNet pour nous créer une baseline de comparaison, mais également pour comparer les performances avec la partie vision.

## Imports de base

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

In [2]:
import sys
from pathlib import Path

# Pour que notre archi fonctionne avec google colab 
!git clone https://github.com/julietteabalain-cloud/Reconnaissance-de-mouvement-artistique
%cd Reconnaissance-de-mouvement-artistique
import sys
sys.path.append(".")  # pour que src/ soit importable

PROJECT_ROOT = Path().resolve().parent
sys.path.append(str(PROJECT_ROOT))

fatal: destination path 'Reconnaissance-de-mouvement-artistique' already exists and is not an empty directory.
/content/Reconnaissance-de-mouvement-artistique


In [3]:
from src.dataset_dl import ArtDataset
from src.train import train_model

from src.dataset import load_df_train_test_val, load_df
from src.preprocessing import clean_dataset

from src.models import get_resnet18
from src.evaluate import compute_confusion_matrix, plot_confusion_matrix, accuracy_per_class, visualize_accuracy_per_style
from src.utils import set_seed

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)



cuda


In [4]:
from google.colab import drive
drive.mount('/content/drive')
from pathlib import Path

PROJECT_ROOT = Path("/content/deepl-projet")
DATA_ROOT = Path("/content/drive/MyDrive/DeepLearning/WikiArt_Subset")


df_test, df_train, df_val = load_df_train_test_val(DATA_ROOT)
df = load_df(DATA_ROOT)

df, df_train, df_val, df_test = clean_dataset(df, df_train, df_val, df_test)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Partie Classique / Baseline

### 1.1 Préparation du dataset

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [6]:
IMAGE_ROOT_TRAIN = DATA_ROOT / "train"
IMAGE_ROOT_VAL = DATA_ROOT / "val"

train_dataset = ArtDataset(
    df_train,
    IMAGE_ROOT_TRAIN,
    transform=transform
)

val_dataset = ArtDataset(
    df_val,
    IMAGE_ROOT_VAL,
    transform=transform
)


In [7]:
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False
)


### 1.2 Charger le modèle

In [8]:
num_classes = len(df_train["style"].unique())

model = get_resnet18(num_classes=num_classes, device=device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(
    model.fc.parameters(),
    lr=1e-3
)


### 1.3 Entrainement du modèle

On commence avec 3 epoch et on freeze.

In [9]:
history_freeze = train_model(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    num_epochs=3
)



Epoch 1/3


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/DeepLearning/WikiArt_Subset/train/5042.jpg'

unfreeze complet :

In [ ]:
for param in model.parameters():
    param.requires_grad = True

optimizer = optim.Adam(
    model.parameters(),
    lr=1e-4
)


On continue avec cette fois 10 epoch.

In [ ]:
history_unfreeze = train_model(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    num_epochs=10
)


### 1.4 Evaluation du modèle

Accuracy (précision)  globale :

In [ ]:
train_acc = history_freeze["train_acc"] + history_unfreeze["train_acc"]
val_acc   = history_freeze["val_acc"]   + history_unfreeze["val_acc"]

plt.plot(train_acc)
plt.plot(val_acc)
plt.legend(["Train", "Validation"])
plt.title("ResNet18 Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.show()


Matrice de confusion :

In [ ]:
#Confusion matrix :

class_names = sorted(df_train["style"].unique())

cm = compute_confusion_matrix(
    model,
    val_loader,
    device,
    class_names
)

plot_confusion_matrix(cm, class_names)


Accuracy par style :

In [ ]:
class_names = sorted(df_train["style"].unique())

acc_per_style = accuracy_per_class(
    model,
    val_loader,
    device,
    class_names
)

results = list(zip(class_names, acc_per_style))
results = sorted(results, key=lambda x: x[1], reverse=True)

for style, acc in results:
    print(f"{style}: {acc:.3f}")


In [ ]:
visualize_accuracy_per_style(results)

### 1.5 Sauvegarde du modèle entrainé

In [ ]:
torch.save(model.state_dict(), "resnet18_baseline.pt")



🔹 Phase B – Préparer l’infrastructure pour ablation

 Créer fonction apply_low_pass(image)

 Créer fonction apply_high_pass(image)

 Version dataset avec filtrage optionnel

 Réentraîner ResNet sur chaque version

 Comparer accuracy globale

 Comparer accuracy par style

 Graphique comparatif final